In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [69]:
df = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv',index_col='id')
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [71]:
df.describe()

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,12.177627,25.924150,20960.281700
std,7.157944,13.624874,22775.459535
min,0.830000,1.000000,1105.000000
25%,6.830000,14.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.080000,38.000000,42521.000000
max,38.580000,49.000000,114523.000000


In [72]:
df.isnull().sum()

airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

# Datasetni modelga tayyorlash

In [73]:
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder()
df['flight']=ordinal.fit_transform(df[['flight']])
df['class'] = ordinal.fit_transform(df[['class']])

In [74]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2,random_state=42)

X=train_set.drop('price',axis=1)
y=train_set['price'].copy()

In [75]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder

num_pipeline = Pipeline([
    ('std_scaler',StandardScaler(),)
])

In [76]:
from sklearn.compose import ColumnTransformer
num_attribs = ['duration','days_left','flight','class']
cat_attribs = ['airline','source_city','destination_city','stops']


full_pipeline = ColumnTransformer([
    ('num',num_pipeline,num_attribs),
    ('cat',OneHotEncoder(),cat_attribs),
])

X_prepared =full_pipeline.fit_transform(X)

# Model  (RandomForestRegressor)

In [77]:
from sklearn.ensemble import RandomForestRegressor
RFR_model = RandomForestRegressor()

RFR_model.fit(X_prepared,y)

RandomForestRegressor()

# Modelni baholash

In [78]:
X = df.drop("price", axis=1)
y = df["price"].copy()

X_prepared = full_pipeline.fit_transform(X)

In [79]:
def display_scores(scores):
    print("Scores:", scores,'\n')
    print("Mean:", scores.mean())

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(RFR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

In [52]:
display_scores(LR_rmse_scores)

Scores: [3376.94558256 3237.92577866 3494.62434203 2954.06382931 3234.75655137
 3236.31474156 3580.45885399 3226.46382927 3165.89318605 3410.98912783] 

Mean: 3291.8435822634992
